In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

In [ ]:
model_weights_path= '/Users/jk1/temp/mimic/prediction/3M_Death/2023_01_04_2020/test_LSTM_sigmoid_all_balanced_0.0_2_True_RMSprop_3M Death_128_2/sigmoid_all_balanced_0.0_2_True_RMSprop_3M Death_128_2.hdf5'
features_path = '/Users/jk1/temp/mimic/preprocessing/mimic_prepro_25012023_232713/preprocessed_features_25012023_232713.csv'
labels_path = '/Users/jk1/temp/mimic/preprocessing/mimic_prepro_25012023_232713/preprocessed_outcomes_25012023_232713.csv'

In [ ]:
outcome = '3M Death'

In [ ]:
model_name = os.path.basename(model_weights_path).split('.hdf5')[0]

model_config = {
    'activation': model_name.split('_')[0],
    'batch': model_name.split('_')[1],
    'data': model_name.split('_')[2],
    'dropout': float(model_name.split('_')[3]),
    'layers': int(model_name.split('_')[4]),
    'masking': model_name.split('_')[5],
    'optimizer': model_name.split('_')[6],
    'units': int(model_name.split('_')[8]),
    'cv_fold': int(model_name.split('_')[9])
}

## Initial calibration curve

Load data

In [ ]:
from prediction.utils.utils import check_data
from prediction.outcome_prediction.data_loading.data_formatting import feature_order_verification, features_to_numpy, \
    format_to_2d_table_with_time

# load the dataset
X, y = format_to_2d_table_with_time(feature_df_path=features_path, outcome_df_path=labels_path,
                                    outcome=outcome)

n_time_steps = X.relative_sample_date_hourly_cat.max() + 1
n_channels = X.sample_label.unique().shape[0]

# test if data is corrupted
check_data(X)

test_X_np = features_to_numpy(X,
                              ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])

# ensure that the order of features (3rd dimension) is the one predefined for the model
feature_order_verification(test_X_np)

test_y_np = np.array([y[y.case_admission_id == cid].outcome.values[0] for cid in
                      test_X_np[:, 0, 0, 0]]).astype('float32')

# Remove the case_admission_id, sample_label, and time_step_label columns from the data
test_X_np = test_X_np[:, :, :, -1].astype('float32')

Load model

In [ ]:
from prediction.outcome_prediction.LSTM.LSTM import lstm_generator
from prediction.utils.scoring import precision, recall, matthews

model = lstm_generator(x_time_shape=n_time_steps, x_channels_shape=n_channels, masking=model_config['masking'], n_units=model_config['units'],
                       activation=model_config['activation'], dropout=model_config['dropout'], n_layers=model_config['layers'])

model.compile(loss='binary_crossentropy', optimizer=model_config['optimizer'],
              metrics=['accuracy', precision, recall, matthews])

model.load_weights(model_weights_path)

Make predictions

In [ ]:
# calculate overall model prediction
y_pred_test = model.predict(test_X_np)

In [ ]:
from prediction.outcome_prediction.LSTM.calibration.calibration_visualisation_tools import plot_calibration_curve

plot_calibration_curve(test_y_np, y_pred_test, n_bins=10)
plt.title("Calibration curve")
plt.show()

## Re-calibration using GSU dataset

In [ ]:
gsu_features_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_features_01012023_233050.csv'
gsu_labels_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_outcomes_01012023_233050.csv'

In [ ]:
from prediction.outcome_prediction.LSTM.calibration.calibration_tools import CalibratableModelFactory

factory = CalibratableModelFactory()

wrapped_model = factory.get_model(model)

In [ ]:
accuracy, MCC = wrapped_model.score(test_X_np, test_y_np)
print(f'{wrapped_model.name} accuracy: {round(100*accuracy, 2)}%')
print(f'MCC: {MCC}')


In [ ]:
plot_calibration_curve(test_y_np, wrapped_model.predict(test_X_np), n_bins=10)
plt.show()

Load calibration data

In [ ]:
from sklearn.model_selection import train_test_split
from prediction.outcome_prediction.data_loading.data_formatting import link_patient_id_to_outcome

# define constants
seed = 42
test_size = 0.20

# load the dataset
X, y = format_to_2d_table_with_time(feature_df_path=gsu_features_path, outcome_df_path=gsu_labels_path,
                                    outcome=outcome)

n_time_steps = X.relative_sample_date_hourly_cat.max() + 1
n_channels = X.sample_label.unique().shape[0]

# test if data is corrupted
check_data(X)

"""
    SPLITTING DATA
    Splitting is done by patient id (and not admission id) as in case of the rare multiple admissions per patient there
    would be a risk of data leakage otherwise split 'pid' in TRAIN and TEST pid = unique patient_id
    """
# Reduce every patient to a single outcome (to avoid duplicates)
all_pids_with_outcome = link_patient_id_to_outcome(y, outcome)
pid_train, pid_test, y_pid_train, y_pid_test = train_test_split(all_pids_with_outcome.patient_id.tolist(),
                                                                all_pids_with_outcome.outcome.tolist(),
                                                                stratify=all_pids_with_outcome.outcome.tolist(),
                                                                test_size=test_size,
                                                                random_state=seed)

calib_X_df = X[X.patient_id.isin(pid_test)]
calib_y_df = y[y.patient_id.isin(pid_test)]

calib_X_np = features_to_numpy(calib_X_df,
                              ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])
calib_y_np = np.array([calib_y_df[calib_y_df.case_admission_id == cid].outcome.values[0] for cid in
                      calib_X_np[:, 0, 0, 0]]).astype('float32')

# Remove the case_admission_id, sample_label, and time_step_label columns from the data
calib_X_np = calib_X_np[:, :, :, -1].astype('float32')

In [ ]:
wrapped_model.calibrate(calib_X_np, calib_y_np)

In [ ]:
for method in ['sigmoid', 'isotonic']:
    accuracy, MCC = wrapped_model.score_calibrated(test_X_np, test_y_np, method)
    print(f'Accuracy after {method}: {round(100*accuracy, 2)}%')
    print(f'MCC after {method}: {MCC}')
    plt.figure(figsize=(10, 4))
    plot_calibration_curve(test_y_np, wrapped_model.predict_calibrated(test_X_np, method))
    plt.title(f'{method} calibration')
    plt.show()

## Re-calibration using a fraction of MIMIC dataset

Gist: use a sub-fraction of MIMIC to calibrate model

In [ ]:
calib_size = 0.1

ext_test_X, ext_calib_X, ext_test_y, ext_calib_y = train_test_split(test_X_np, test_y_np,
                                                                    stratify=test_y_np,
                                                                test_size=calib_size,
                                                                random_state=seed)

In [ ]:
wrapped_model.calibrate(ext_calib_X, ext_calib_y)

In [ ]:
for method in ['sigmoid', 'isotonic']:
    accuracy, MCC = wrapped_model.score_calibrated(test_X_np, test_y_np, method)
    print(f'Accuracy after {method}: {round(100*accuracy, 2)}%')
    print(f'MCC after {method}: {MCC}')
    plt.figure(figsize=(10, 4))
    plot_calibration_curve(test_y_np, wrapped_model.predict_calibrated(test_X_np, method))
    plt.title(f'{method} calibration')
    plt.show()